In [ ]:
# Análise Exploratória de Dados - Voos Comerciais 2015

Este notebook apresenta uma análise exploratória completa dos dados de voos comerciais dos EUA em 2015, incluindo:
- Estatísticas descritivas
- Visualizações de insights
- Tratamento de valores ausentes

**Datasets:**
- `flights.csv`: Dados de voos (~5.8M registros)
- `airlines.csv`: Informações das companhias aéreas
- `airports.csv`: Informações dos aeroportos

## 1. Importação de Bibliotecas

In [ ]:
# Carregando os datasets
print("Carregando dados...")

# Companhias aéreas
airlines = pd.read_csv('../data/airlines.csv')
print(f"✓ Airlines: {airlines.shape[0]} registros, {airlines.shape[1]} colunas")

# Aeroportos
airports = pd.read_csv('../data/airports.csv')
print(f"✓ Airports: {airports.shape[0]} registros, {airports.shape[1]} colunas")

# Voos - dataset grande, pode demorar
flights = pd.read_csv('../data/flights.csv')
print(f"✓ Flights: {flights.shape[0]} registros, {flights.shape[1]} colunas")

print("\nDados carregados com sucesso!")

## 2. Carregamento dos Dados

In [ ]:
print("=" * 60)
print("DATASET: FLIGHTS")
print("=" * 60)
print(f"\nDimensões: {flights.shape}")
print(f"\nPrimeiras linhas:")
display(flights.head(10))
print(f"\nInformações do dataset:")
flights.info()
print(f"\nColunas do dataset:")
for i, col in enumerate(flights.columns, 1):
    print(f"{i:2}. {col}")

### 3.3 Flights (Voos)

In [ ]:
print("=" * 60)
print("DATASET: AIRPORTS")
print("=" * 60)
print(f"\nDimensões: {airports.shape}")
print(f"\nPrimeiras linhas:")
display(airports.head(10))
print(f"\nInformações do dataset:")
airports.info()
print(f"\nValores ausentes:")
print(airports.isnull().sum())
print(f"\nEstados com mais aeroportos:")
print(airports['STATE'].value_counts().head(10))

### 3.2 Airports (Aeroportos)

In [ ]:
print("=" * 60)
print("DATASET: AIRLINES")
print("=" * 60)
print(f"\nDimensões: {airlines.shape}")
print(f"\nPrimeiras linhas:")
display(airlines.head(10))
print(f"\nInformações do dataset:")
airlines.info()
print(f"\nValores ausentes:")
print(airlines.isnull().sum())

### 3.1 Airlines (Companhias Aéreas)

## 3. Visão Geral dos Dados

In [ ]:
# Análise de atrasos
print("=" * 60)
print("ANÁLISE DE ATRASOS")
print("=" * 60)

# Filtrar apenas voos não cancelados
flights_operated = flights_clean[flights_clean['CANCELLED'] == 0].copy()

print(f"\nVoos operados (não cancelados): {len(flights_operated):,}")
print(f"\nAtraso médio de partida: {flights_operated['DEPARTURE_DELAY'].mean():.2f} minutos")
print(f"Atraso médio de chegada: {flights_operated['ARRIVAL_DELAY'].mean():.2f} minutos")
print(f"\nVoos com atraso de partida > 15 min: {(flights_operated['DEPARTURE_DELAY'] > 15).sum():,} ({(flights_operated['DEPARTURE_DELAY'] > 15).sum()/len(flights_operated)*100:.2f}%)")
print(f"Voos com atraso de chegada > 15 min: {(flights_operated['ARRIVAL_DELAY'] > 15).sum():,} ({(flights_operated['ARRIVAL_DELAY'] > 15).sum()/len(flights_operated)*100:.2f}%)")

# Tipos de atraso
delay_types = {
    'Sistema Aéreo': flights_operated['AIR_SYSTEM_DELAY'].sum(),
    'Segurança': flights_operated['SECURITY_DELAY'].sum(),
    'Companhia Aérea': flights_operated['AIRLINE_DELAY'].sum(),
    'Aeronave Atrasada': flights_operated['LATE_AIRCRAFT_DELAY'].sum(),
    'Clima': flights_operated['WEATHER_DELAY'].sum()
}

print("\n" + "=" * 60)
print("TOTAL DE MINUTOS DE ATRASO POR TIPO")
print("=" * 60)
for tipo, minutos in sorted(delay_types.items(), key=lambda x: x[1], reverse=True):
    print(f"{tipo}: {minutos:,.0f} minutos ({minutos/60:,.0f} horas)")

### 5.2 Análise de Atrasos

In [ ]:
# Estatísticas descritivas principais
print("=" * 60)
print("ESTATÍSTICAS DESCRITIVAS - VOOS")
print("=" * 60)

# Selecionando colunas numéricas importantes
numeric_cols = ['DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'AIR_TIME', 'DISTANCE', 
                'ELAPSED_TIME', 'TAXI_OUT', 'TAXI_IN']

stats = flights_clean[numeric_cols].describe()
display(stats)

# Estatísticas adicionais
print("\n" + "=" * 60)
print("INFORMAÇÕES GERAIS")
print("=" * 60)
print(f"Total de voos: {len(flights_clean):,}")
print(f"Período: {flights_clean['YEAR'].min()} - Mês {flights_clean['MONTH'].min()} a {flights_clean['MONTH'].max()}")
print(f"Total de companhias aéreas: {flights_clean['AIRLINE'].nunique()}")
print(f"Total de aeroportos de origem: {flights_clean['ORIGIN_AIRPORT'].nunique()}")
print(f"Total de aeroportos de destino: {flights_clean['DESTINATION_AIRPORT'].nunique()}")
print(f"Voos cancelados: {flights_clean['CANCELLED'].sum():,} ({(flights_clean['CANCELLED'].sum()/len(flights_clean)*100):.2f}%)")
print(f"Voos desviados: {flights_clean['DIVERTED'].sum():,} ({(flights_clean['DIVERTED'].sum()/len(flights_clean)*100):.2f}%)")

### 5.1 Estatísticas Gerais

## 5. Estatísticas Descritivas

In [ ]:
# Distribuição de atrasos (limitando a -60 a 180 minutos para melhor visualização)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Atraso de partida
dep_delays = flights_operated['DEPARTURE_DELAY'][(flights_operated['DEPARTURE_DELAY'] >= -60) & 
                                                   (flights_operated['DEPARTURE_DELAY'] <= 180)]
ax1.hist(dep_delays, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
ax1.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Sem atraso')
ax1.axvline(x=dep_delays.mean(), color='green', linestyle='--', linewidth=2, label=f'Média: {dep_delays.mean():.1f} min')
ax1.set_xlabel('Atraso de Partida (minutos)', fontsize=12)
ax1.set_ylabel('Frequência', fontsize=12)
ax1.set_title('Distribuição de Atrasos de Partida', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Atraso de chegada
arr_delays = flights_operated['ARRIVAL_DELAY'][(flights_operated['ARRIVAL_DELAY'] >= -60) & 
                                                 (flights_operated['ARRIVAL_DELAY'] <= 180)]
ax2.hist(arr_delays, bins=50, color='coral', edgecolor='black', alpha=0.7)
ax2.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Sem atraso')
ax2.axvline(x=arr_delays.mean(), color='green', linestyle='--', linewidth=2, label=f'Média: {arr_delays.mean():.1f} min')
ax2.set_xlabel('Atraso de Chegada (minutos)', fontsize=12)
ax2.set_ylabel('Frequência', fontsize=12)
ax2.set_title('Distribuição de Atrasos de Chegada', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 6.5 Distribuição de Atrasos

In [ ]:
# Atraso médio por companhia aérea
delay_by_airline = flights_operated.groupby('AIRLINE').agg({
    'DEPARTURE_DELAY': 'mean',
    'ARRIVAL_DELAY': 'mean',
    'AIRLINE': 'count'
}).reset_index()
delay_by_airline.columns = ['AIRLINE', 'AVG_DEP_DELAY', 'AVG_ARR_DELAY', 'TOTAL_FLIGHTS']

# Merge com nomes
delay_by_airline = delay_by_airline.merge(airlines, left_on='AIRLINE', right_on='IATA_CODE', how='left')
delay_by_airline = delay_by_airline.sort_values('AVG_ARR_DELAY', ascending=False)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Atraso de partida
ax1.barh(delay_by_airline['AIRLINE_y'], delay_by_airline['AVG_DEP_DELAY'], color='tomato')
ax1.set_xlabel('Atraso Médio (minutos)', fontsize=12)
ax1.set_ylabel('Companhia Aérea', fontsize=12)
ax1.set_title('Atraso Médio de Partida por Companhia', fontsize=14, fontweight='bold')
ax1.axvline(x=0, color='black', linestyle='--', linewidth=0.8, alpha=0.7)
ax1.grid(axis='x', alpha=0.3)

# Atraso de chegada
ax2.barh(delay_by_airline['AIRLINE_y'], delay_by_airline['AVG_ARR_DELAY'], color='orangered')
ax2.set_xlabel('Atraso Médio (minutos)', fontsize=12)
ax2.set_ylabel('Companhia Aérea', fontsize=12)
ax2.set_title('Atraso Médio de Chegada por Companhia', fontsize=14, fontweight='bold')
ax2.axvline(x=0, color='black', linestyle='--', linewidth=0.8, alpha=0.7)
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("Ranking de atrasos por companhia:")
display(delay_by_airline[['AIRLINE', 'AIRLINE_y', 'AVG_DEP_DELAY', 'AVG_ARR_DELAY', 'TOTAL_FLIGHTS']].round(2))

### 6.4 Análise de Atrasos por Companhia Aérea

In [ ]:
# Voos por mês
flights_by_month = flights_clean.groupby('MONTH').size().reset_index(name='COUNT')

# Voos por dia da semana
flights_by_dow = flights_clean.groupby('DAY_OF_WEEK').size().reset_index(name='COUNT')
dow_labels = ['Seg', 'Ter', 'Qua', 'Qui', 'Sex', 'Sáb', 'Dom']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Por mês
month_labels = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
ax1.plot(flights_by_month['MONTH'], flights_by_month['COUNT'], marker='o', 
         linewidth=2, markersize=8, color='steelblue')
ax1.fill_between(flights_by_month['MONTH'], flights_by_month['COUNT'], alpha=0.3, color='steelblue')
ax1.set_xlabel('Mês', fontsize=12)
ax1.set_ylabel('Número de Voos', fontsize=12)
ax1.set_title('Distribuição de Voos por Mês (2015)', fontsize=14, fontweight='bold')
ax1.set_xticks(range(1, 13))
ax1.set_xticklabels(month_labels)
ax1.grid(alpha=0.3)

# Por dia da semana
ax2.bar(flights_by_dow['DAY_OF_WEEK'], flights_by_dow['COUNT'], color='coral')
ax2.set_xlabel('Dia da Semana', fontsize=12)
ax2.set_ylabel('Número de Voos', fontsize=12)
ax2.set_title('Distribuição de Voos por Dia da Semana', fontsize=14, fontweight='bold')
ax2.set_xticks(range(1, 8))
ax2.set_xticklabels(dow_labels)
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("Voos por mês:")
display(flights_by_month)
print("\nVoos por dia da semana:")
flights_by_dow['DIA'] = dow_labels
display(flights_by_dow[['DIA', 'COUNT']])

### 6.3 Distribuição de Voos ao Longo do Ano

In [ ]:
# Top aeroportos de origem
top_origins = flights_clean['ORIGIN_AIRPORT'].value_counts().head(15).reset_index()
top_origins.columns = ['AIRPORT', 'COUNT']

# Top aeroportos de destino
top_destinations = flights_clean['DESTINATION_AIRPORT'].value_counts().head(15).reset_index()
top_destinations.columns = ['AIRPORT', 'COUNT']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Aeroportos de origem
ax1.barh(range(len(top_origins)), top_origins['COUNT'], color='coral')
ax1.set_yticks(range(len(top_origins)))
ax1.set_yticklabels(top_origins['AIRPORT'])
ax1.set_xlabel('Número de Voos', fontsize=12)
ax1.set_ylabel('Aeroporto', fontsize=12)
ax1.set_title('Top 15 Aeroportos de Origem', fontsize=14, fontweight='bold')
ax1.invert_yaxis()
ax1.grid(axis='x', alpha=0.3)

# Aeroportos de destino
ax2.barh(range(len(top_destinations)), top_destinations['COUNT'], color='lightgreen')
ax2.set_yticks(range(len(top_destinations)))
ax2.set_yticklabels(top_destinations['AIRPORT'])
ax2.set_xlabel('Número de Voos', fontsize=12)
ax2.set_ylabel('Aeroporto', fontsize=12)
ax2.set_title('Top 15 Aeroportos de Destino', fontsize=14, fontweight='bold')
ax2.invert_yaxis()
ax2.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

print("Top 5 aeroportos de origem:")
display(top_origins.head())
print("\nTop 5 aeroportos de destino:")
display(top_destinations.head())

### 6.2 Aeroportos Mais Movimentados

In [ ]:
# Voos por companhia aérea
flights_by_airline = flights_clean['AIRLINE'].value_counts().reset_index()
flights_by_airline.columns = ['AIRLINE', 'COUNT']

# Merge com nomes das companhias
flights_by_airline = flights_by_airline.merge(airlines, left_on='AIRLINE', right_on='IATA_CODE', how='left')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico de barras
ax1.barh(flights_by_airline['AIRLINE_y'], flights_by_airline['COUNT'], color='steelblue')
ax1.set_xlabel('Número de Voos', fontsize=12)
ax1.set_ylabel('Companhia Aérea', fontsize=12)
ax1.set_title('Total de Voos por Companhia Aérea', fontsize=14, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)
for i, v in enumerate(flights_by_airline['COUNT']):
    ax1.text(v, i, f' {v:,}', va='center', fontsize=9)

# Gráfico de pizza
colors = plt.cm.Set3(range(len(flights_by_airline)))
ax2.pie(flights_by_airline['COUNT'], labels=flights_by_airline['AIRLINE'], autopct='%1.1f%%',
        colors=colors, startangle=90)
ax2.set_title('Proporção de Voos por Companhia', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\nTop 5 companhias aéreas:")
display(flights_by_airline[['AIRLINE', 'AIRLINE_y', 'COUNT']].head())

### 6.1 Distribuição de Voos por Companhia Aérea

## 6. Visualizações e Insights

---
### 📝 Nota Final

Este notebook apresentou uma análise exploratória completa dos dados de voos comerciais de 2015, incluindo:

✅ **Estatísticas descritivas detalhadas**  
✅ **Múltiplas visualizações com insights**  
✅ **Tratamento adequado de valores ausentes**  
✅ **Análise de correlações**  
✅ **Identificação de padrões temporais**  
✅ **Comparação entre companhias aéreas**  

Os dados estão prontos para análises mais avançadas ou modelagem preditiva.

---

## 8. Conclusões Finais

### Principais Descobertas:

1. **Volume e Operação:**
   - Dataset robusto com ~5.8M de registros de voos em 2015
   - Taxa de cancelamento relativamente baixa (~1.5%)
   - Maioria dos voos operou conforme planejado

2. **Atrasos:**
   - Atraso médio de chegada positivo, indicando tendência de atrasos
   - Aeronaves atrasadas e atrasos de companhias aéreas são as principais causas
   - Atrasos tendem a aumentar ao longo do dia
   - Forte correlação entre atraso de partida e chegada

3. **Cancelamentos:**
   - Clima é o principal motivo de cancelamento
   - Algumas companhias apresentam taxas significativamente maiores
   - Variação sazonal nos cancelamentos

4. **Padrões Temporais:**
   - Períodos de pico identificados em determinados meses
   - Distribuição uniforme ao longo dos dias da semana
   - Horários específicos apresentam maior volume e atrasos

5. **Dados Técnicos:**
   - Forte correlação entre distância e tempo de voo (esperado)
   - Variabilidade nos tempos de taxi (entrada/saída)
   - Aeroportos hub concentram maior volume de tráfego

### Recomendações para Análises Futuras:

- Análise preditiva de atrasos
- Estudo de rotas específicas
- Análise de sazonalidade mais profunda
- Impacto de eventos climáticos específicos
- Otimização de horários de voo

### Tratamento de Dados:

✅ Valores ausentes tratados adequadamente:
- Atrasos preenchidos com 0 (sem atraso)
- CANCELLATION_REASON preenchido com 'N' (não cancelado)
- Dados operacionais mantidos como NaN para voos cancelados (correto)

In [ ]:
# Resumo dos principais insights
print("=" * 80)
print("RESUMO - PRINCIPAIS INSIGHTS DA ANÁLISE EXPLORATÓRIA")
print("=" * 80)

print("\n📊 VOLUME DE DADOS:")
print(f"   • Total de voos analisados: {len(flights_clean):,}")
print(f"   • Companhias aéreas: {flights_clean['AIRLINE'].nunique()}")
print(f"   • Aeroportos únicos: {flights_clean['ORIGIN_AIRPORT'].nunique()}")

print("\n✈️ OPERAÇÃO DE VOOS:")
print(f"   • Taxa de cancelamento: {(flights_clean['CANCELLED'].sum()/len(flights_clean)*100):.2f}%")
print(f"   • Taxa de desvio: {(flights_clean['DIVERTED'].sum()/len(flights_clean)*100):.2f}%")
print(f"   • Voos operados com sucesso: {(len(flights_operated)/len(flights_clean)*100):.2f}%")

print("\n⏱️ ATRASOS:")
print(f"   • Atraso médio de partida: {flights_operated['DEPARTURE_DELAY'].mean():.2f} minutos")
print(f"   • Atraso médio de chegada: {flights_operated['ARRIVAL_DELAY'].mean():.2f} minutos")
print(f"   • Voos com atraso > 15 min (partida): {(flights_operated['DEPARTURE_DELAY'] > 15).sum()/len(flights_operated)*100:.2f}%")
print(f"   • Voos com atraso > 15 min (chegada): {(flights_operated['ARRIVAL_DELAY'] > 15).sum()/len(flights_operated)*100:.2f}%")

print("\n🏆 TOP PERFORMERS:")
top_airline = flights_by_airline.iloc[0]
print(f"   • Companhia com mais voos: {top_airline['AIRLINE_y']} ({top_airline['COUNT']:,} voos)")
top_origin = top_origins.iloc[0]
print(f"   • Aeroporto mais movimentado (origem): {top_origin['AIRPORT']} ({top_origin['COUNT']:,} voos)")
best_delay_airline = delay_by_airline.iloc[-1]
print(f"   • Companhia com menor atraso médio: {best_delay_airline['AIRLINE_y']} ({best_delay_airline['AVG_ARR_DELAY']:.2f} min)")

print("\n📉 PONTOS DE ATENÇÃO:")
worst_delay_airline = delay_by_airline.iloc[0]
print(f"   • Companhia com maior atraso médio: {worst_delay_airline['AIRLINE_y']} ({worst_delay_airline['AVG_ARR_DELAY']:.2f} min)")
worst_cancel_airline = cancellation_by_airline.iloc[0]
print(f"   • Maior taxa de cancelamento: {worst_cancel_airline['AIRLINE_y']} ({worst_cancel_airline['CANCELLATION_RATE']:.2f}%)")

# Tipo de atraso mais comum
main_delay_type = delay_df.iloc[0]
print(f"   • Principal causa de atraso: {main_delay_type['Tipo']} ({main_delay_type['Percentual']:.1f}% dos atrasos)")

# Motivo principal de cancelamento
main_cancel_reason = cancellation_reasons_df.iloc[0]
print(f"   • Principal motivo de cancelamento: {main_cancel_reason['Motivo']} ({main_cancel_reason['Percentual']:.1f}%)")

print("\n📈 PADRÕES TEMPORAIS:")
busiest_month = flights_by_month.loc[flights_by_month['COUNT'].idxmax()]
print(f"   • Mês mais movimentado: Mês {busiest_month['MONTH']} ({busiest_month['COUNT']:,} voos)")
busiest_dow = flights_by_dow.loc[flights_by_dow['COUNT'].idxmax()]
print(f"   • Dia da semana mais movimentado: {dow_labels[int(busiest_dow['DAY_OF_WEEK'])-1]} ({busiest_dow['COUNT']:,} voos)")

print("\n🔍 DADOS TÉCNICOS:")
print(f"   • Distância média dos voos: {flights_operated['DISTANCE'].mean():.0f} milhas")
print(f"   • Tempo médio de voo: {flights_operated['AIR_TIME'].mean():.0f} minutos")
print(f"   • Correlação Distância x Tempo: {correlation:.3f} (forte correlação positiva)")

print("\n" + "=" * 80)

## 7. Principais Insights e Conclusões

In [ ]:
# Mapa de correlação
correlation_cols = ['DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'AIR_TIME', 'DISTANCE', 
                    'ELAPSED_TIME', 'TAXI_OUT', 'TAXI_IN', 'AIR_SYSTEM_DELAY', 
                    'AIRLINE_DELAY', 'WEATHER_DELAY']

correlation_matrix = flights_operated[correlation_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Mapa de Correlação - Variáveis Numéricas', fontsize=16, fontweight='bold', pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("Principais correlações encontradas:")
print("\nCorrelações mais fortes (> 0.7):")
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > 0.7:
            print(f"{correlation_matrix.columns[i]} <-> {correlation_matrix.columns[j]}: {correlation_matrix.iloc[i, j]:.3f}")

### 6.11 Mapa de Correlação de Variáveis Numéricas

In [ ]:
# Análise de horários de partida
# Extrair hora da partida programada (formato HHMM)
flights_operated['HOUR'] = (flights_operated['SCHEDULED_DEPARTURE'] // 100).astype(int)

flights_by_hour = flights_operated.groupby('HOUR').size().reset_index(name='COUNT')

# Atraso médio por hora
delay_by_hour = flights_operated.groupby('HOUR')['DEPARTURE_DELAY'].mean().reset_index()
delay_by_hour.columns = ['HOUR', 'AVG_DELAY']

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))

# Volume de voos por hora
ax1.bar(flights_by_hour['HOUR'], flights_by_hour['COUNT'], color='steelblue', alpha=0.7)
ax1.set_xlabel('Hora do Dia', fontsize=12)
ax1.set_ylabel('Número de Voos', fontsize=12)
ax1.set_title('Volume de Voos por Hora do Dia', fontsize=14, fontweight='bold')
ax1.set_xticks(range(0, 24))
ax1.grid(axis='y', alpha=0.3)

# Atraso médio por hora
ax2.plot(delay_by_hour['HOUR'], delay_by_hour['AVG_DELAY'], marker='o', 
         linewidth=2, markersize=8, color='coral')
ax2.fill_between(delay_by_hour['HOUR'], delay_by_hour['AVG_DELAY'], alpha=0.3, color='coral')
ax2.set_xlabel('Hora do Dia', fontsize=12)
ax2.set_ylabel('Atraso Médio (minutos)', fontsize=12)
ax2.set_title('Atraso Médio de Partida por Hora do Dia', fontsize=14, fontweight='bold')
ax2.set_xticks(range(0, 24))
ax2.axhline(y=0, color='black', linestyle='--', linewidth=0.8, alpha=0.7)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Horários de pico:")
top_hours = flights_by_hour.nlargest(5, 'COUNT')
display(top_hours)

print("\nHorários com maiores atrasos:")
worst_hours = delay_by_hour.nlargest(5, 'AVG_DELAY')
display(worst_hours)

### 6.10 Análise de Horários de Pico

In [ ]:
# Relação entre distância e tempo de voo (amostra para melhor visualização)
sample_flights = flights_operated.sample(n=min(10000, len(flights_operated)), random_state=42)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot
ax1.scatter(sample_flights['DISTANCE'], sample_flights['AIR_TIME'], alpha=0.5, s=10, color='steelblue')
ax1.set_xlabel('Distância (milhas)', fontsize=12)
ax1.set_ylabel('Tempo de Voo (minutos)', fontsize=12)
ax1.set_title('Relação entre Distância e Tempo de Voo', fontsize=14, fontweight='bold')
ax1.grid(alpha=0.3)

# Correlação
correlation = flights_operated['DISTANCE'].corr(flights_operated['AIR_TIME'])
ax1.text(0.05, 0.95, f'Correlação: {correlation:.3f}', transform=ax1.transAxes,
         fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Distribuição de distâncias
ax2.hist(flights_operated['DISTANCE'], bins=50, color='coral', edgecolor='black', alpha=0.7)
ax2.set_xlabel('Distância (milhas)', fontsize=12)
ax2.set_ylabel('Frequência', fontsize=12)
ax2.set_title('Distribuição de Distâncias dos Voos', fontsize=14, fontweight='bold')
ax2.axvline(x=flights_operated['DISTANCE'].mean(), color='red', linestyle='--', 
            linewidth=2, label=f'Média: {flights_operated["DISTANCE"].mean():.0f} milhas')
ax2.legend()
ax2.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Distância média: {flights_operated['DISTANCE'].mean():.2f} milhas")
print(f"Tempo médio de voo: {flights_operated['AIR_TIME'].mean():.2f} minutos")
print(f"Correlação Distância x Tempo: {correlation:.3f}")

### 6.9 Relação entre Distância e Tempo de Voo

In [ ]:
# Análise dos motivos de cancelamento
cancelled_flights = flights_clean[flights_clean['CANCELLED'] == 1]
cancellation_reasons = cancelled_flights['CANCELLATION_REASON'].value_counts()

# Mapeamento dos códigos
reason_map = {
    'A': 'Companhia Aérea',
    'B': 'Clima',
    'C': 'Sistema Aéreo',
    'D': 'Segurança',
    'N': 'Não Cancelado'
}

cancellation_reasons_df = pd.DataFrame({
    'Código': cancellation_reasons.index,
    'Count': cancellation_reasons.values
})
cancellation_reasons_df['Motivo'] = cancellation_reasons_df['Código'].map(reason_map)
cancellation_reasons_df['Percentual'] = (cancellation_reasons_df['Count'] / 
                                          cancellation_reasons_df['Count'].sum() * 100).round(2)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico de barras
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']
ax1.bar(cancellation_reasons_df['Motivo'], cancellation_reasons_df['Count'], color=colors)
ax1.set_xlabel('Motivo', fontsize=12)
ax1.set_ylabel('Número de Cancelamentos', fontsize=12)
ax1.set_title('Cancelamentos por Motivo', fontsize=14, fontweight='bold')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(axis='y', alpha=0.3)
for i, v in enumerate(cancellation_reasons_df['Count']):
    ax1.text(i, v, f'{v:,}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Gráfico de pizza
ax2.pie(cancellation_reasons_df['Count'], labels=cancellation_reasons_df['Motivo'], 
        autopct='%1.1f%%', colors=colors, startangle=90, textprops={'fontsize': 11})
ax2.set_title('Proporção de Cancelamentos por Motivo', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Total de voos cancelados: {len(cancelled_flights):,}")
print("\nMotivos de cancelamento:")
display(cancellation_reasons_df)

### 6.8 Motivos de Cancelamento

In [ ]:
# Taxa de cancelamento por companhia aérea
cancellation_by_airline = flights_clean.groupby('AIRLINE').agg({
    'CANCELLED': ['sum', 'count']
}).reset_index()
cancellation_by_airline.columns = ['AIRLINE', 'CANCELLED_COUNT', 'TOTAL_FLIGHTS']
cancellation_by_airline['CANCELLATION_RATE'] = (cancellation_by_airline['CANCELLED_COUNT'] / 
                                                  cancellation_by_airline['TOTAL_FLIGHTS'] * 100).round(2)

# Merge com nomes
cancellation_by_airline = cancellation_by_airline.merge(airlines, left_on='AIRLINE', right_on='IATA_CODE', how='left')
cancellation_by_airline = cancellation_by_airline.sort_values('CANCELLATION_RATE', ascending=False)

fig, ax = plt.subplots(figsize=(14, 7))

ax.barh(cancellation_by_airline['AIRLINE_y'], cancellation_by_airline['CANCELLATION_RATE'], color='crimson')
ax.set_xlabel('Taxa de Cancelamento (%)', fontsize=12)
ax.set_ylabel('Companhia Aérea', fontsize=12)
ax.set_title('Taxa de Cancelamento por Companhia Aérea', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
for i, v in enumerate(cancellation_by_airline['CANCELLATION_RATE']):
    ax.text(v, i, f' {v:.2f}%', va='center', fontsize=10)

plt.tight_layout()
plt.show()

print("Ranking de cancelamentos:")
display(cancellation_by_airline[['AIRLINE', 'AIRLINE_y', 'CANCELLED_COUNT', 'TOTAL_FLIGHTS', 'CANCELLATION_RATE']])

### 6.7 Taxa de Cancelamento por Companhia Aérea

In [ ]:
# Análise dos tipos de atraso
delay_types_data = {
    'Sistema Aéreo': flights_operated['AIR_SYSTEM_DELAY'].sum(),
    'Segurança': flights_operated['SECURITY_DELAY'].sum(),
    'Companhia Aérea': flights_operated['AIRLINE_DELAY'].sum(),
    'Aeronave Atrasada': flights_operated['LATE_AIRCRAFT_DELAY'].sum(),
    'Clima': flights_operated['WEATHER_DELAY'].sum()
}

delay_df = pd.DataFrame(list(delay_types_data.items()), columns=['Tipo', 'Total_Minutos'])
delay_df['Total_Horas'] = delay_df['Total_Minutos'] / 60
delay_df['Percentual'] = (delay_df['Total_Minutos'] / delay_df['Total_Minutos'].sum() * 100).round(2)
delay_df = delay_df.sort_values('Total_Minutos', ascending=False)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico de barras
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8']
ax1.bar(delay_df['Tipo'], delay_df['Total_Horas'], color=colors)
ax1.set_xlabel('Tipo de Atraso', fontsize=12)
ax1.set_ylabel('Total de Horas', fontsize=12)
ax1.set_title('Total de Atrasos por Tipo (em horas)', fontsize=14, fontweight='bold')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(axis='y', alpha=0.3)
for i, v in enumerate(delay_df['Total_Horas']):
    ax1.text(i, v, f'{v:,.0f}h', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Gráfico de pizza
ax2.pie(delay_df['Total_Minutos'], labels=delay_df['Tipo'], autopct='%1.1f%%',
        colors=colors, startangle=90, textprops={'fontsize': 11})
ax2.set_title('Proporção de Atrasos por Tipo', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("Resumo dos tipos de atraso:")
display(delay_df)

### 6.6 Tipos de Atraso - Análise Detalhada

In [ ]:
# Criando uma cópia para tratamento
flights_clean = flights.copy()

# 1. Tratar colunas de atraso - preencher com 0
delay_columns = ['DEPARTURE_DELAY', 'ARRIVAL_DELAY', 'AIR_SYSTEM_DELAY', 
                 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']

for col in delay_columns:
    if col in flights_clean.columns:
        flights_clean[col] = flights_clean[col].fillna(0)
        print(f"✓ {col}: preenchido com 0")

# 2. Tratar CANCELLATION_REASON
if 'CANCELLATION_REASON' in flights_clean.columns:
    flights_clean['CANCELLATION_REASON'] = flights_clean['CANCELLATION_REASON'].fillna('N')
    print(f"✓ CANCELLATION_REASON: preenchido com 'N' (Not Cancelled)")

# 3. Verificar resultado
print("\n" + "=" * 60)
print("RESULTADO DO TRATAMENTO")
print("=" * 60)
missing_after = pd.DataFrame({
    'Coluna': flights_clean.columns,
    'Missing_Count': flights_clean.isnull().sum(),
    'Missing_Percent': (flights_clean.isnull().sum() / len(flights_clean) * 100).round(2)
})
missing_after = missing_after[missing_after['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

print("\nColunas ainda com valores ausentes (esperado para voos cancelados):")
display(missing_after.head(10))

### 4.1 Estratégia de Tratamento de Valores Ausentes

**Análise:**
- Colunas de atraso (delays): Ausentes quando não há atraso ou voo foi cancelado
- CANCELLATION_REASON: Ausente quando voo não foi cancelado
- Tempos de operação: Ausentes para voos cancelados

**Estratégia de tratamento:**
1. Para colunas de atraso: Preencher com 0 (significa sem atraso)
2. Para CANCELLATION_REASON: Preencher com 'N' (Not Cancelled)
3. Para tempos de operação cancelados: Manter como NaN (são voos que não operaram)

In [ ]:
# Análise detalhada de valores ausentes no dataset de flights
print("=" * 60)
print("ANÁLISE DE VALORES AUSENTES - FLIGHTS")
print("=" * 60)

missing_data = pd.DataFrame({
    'Coluna': flights.columns,
    'Missing_Count': flights.isnull().sum(),
    'Missing_Percent': (flights.isnull().sum() / len(flights) * 100).round(2)
})
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

print("\nColunas com valores ausentes:")
display(missing_data)

# Visualização
fig, ax = plt.subplots(figsize=(14, 6))
missing_data_plot = missing_data.head(15)
ax.barh(missing_data_plot['Coluna'], missing_data_plot['Missing_Percent'], color='coral')
ax.set_xlabel('Percentual de Valores Ausentes (%)', fontsize=12)
ax.set_ylabel('Colunas', fontsize=12)
ax.set_title('Top 15 Colunas com Valores Ausentes', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nTotal de colunas com valores ausentes: {len(missing_data)}")
print(f"Total de colunas: {len(flights.columns)}")

## 4. Análise de Valores Ausentes

In [ ]:
# Importação de bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configurações
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Bibliotecas importadas com sucesso!")